# Building a Spam Filter with Naive Bayes

We'll use the multinomial Naive Bayes algorithm along with a dataset of 5,572 SMS messages that are already classified by humans to "teach" the computer how to classify messages.

The dataset was put together by Tiago A. Almeida and José María Gómez Hidalgo, and it can be downloaded from the [The UCI Machine Learning Repository](https://archive.ics.uci.edu/ml/datasets/sms+spam+collection). The data collection process is described in more details [on this page](http://www.dt.fee.unicamp.br/~tiago/smsspamcollection/#composition)

The goal of this project is building a spam filter for SMS messages.


## Exploring the Dataset

In [1]:
import pandas as pd

data = pd.read_csv('~/Documents/Python/SMSSpamCollection', header=None,sep='\t', names=['Label', 'SMS'])

In [2]:
data.head(10)

,Label,SMS
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."
5,spam,FreeMsg Hey there darling it's been 3 week's n...
6,ham,Even my brother is not like to speak with me. ...
7,ham,As per your request 'Melle Melle (Oru Minnamin...
8,spam,WINNER!! As a valued network customer you have...
9,spam,Had your mobile 11 months or more? U R entitle...


In [3]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Label   5572 non-null   object
 1   SMS     5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


In [4]:
label = data['Label']
label.value_counts()

ham     4825
spam     747
Name: Label, dtype: int64

In [5]:
# let's find percentage of spam messages and non-spam messages:
pct_spam = 747/5572*100
print('Percentage of the spam messages is {:.2f}%'.format(pct_spam))

Percentage of the spam messages is 13.41%


In [6]:
pct_non_spam = 4825/5572*100
print('Percentage of the non_spam messages is {:.2f}%'.format(pct_non_spam))

Percentage of the non_spam messages is 86.59%


## Training and Test Set

To test the spam filter, I'm first going to split the dataset into two categories:
- A training set, which I'll use to "train" the computer how to classify messages.
- A test set, which I'll use to test how good the spam filter is with classifying new messages.

I'm going to keep 80% of dataset for training, and 20% for testing (I want to train the algorithm on as much data as possible, but I also want to have enough test data). The dataset has 5,572 messages, which means that:

- The training set will have 4,458 messages (about 80% of the dataset).
- The test set will have 1,114 messages (about 20% of the dataset)


In [7]:
# Randomize the dataset
data_random = data.sample(frac=1, random_state=1)

In [8]:
# Calculate index for split
training_test_index = round(len(data_random) * 0.8)

# Training/Test split
training_set = data_random[:training_test_index].reset_index(drop=True)
test_set = data_random[training_test_index:].reset_index(drop=True)

print(training_set.shape)
print(test_set.shape)

(4458, 2)
(1114, 2)


In [9]:
# find the number of spam messages and non-spam messages in both new data sets
training_set['Label'].value_counts()

ham     3858
spam     600
Name: Label, dtype: int64

In [10]:
test_set['Label'].value_counts()

ham     967
spam    147
Name: Label, dtype: int64

In [11]:
# Find the percentage of spam and ham in the training 
pct_non_spam_training_set = 3858/4458*100
print('Percentage of the non-spam messages in training_set is {:.2f}%'.format(pct_non_spam_training_set))

pct_spam_training_set = 600/4458*100
print('Percentage of the spam messages in training_set is {:.2f}%'.format(pct_spam_training_set))


Percentage of the non-spam messages in training_set is 86.54%
Percentage of the spam messages in training_set is 13.46%


In [12]:
# Find the percentage of spam and ham in the test set

pct_non_spam_test_set = 967/1114*100
print('Percentage of the non-spam messages in test_set is {:.2f}%'.format(pct_non_spam_test_set))

pct_spam_test_set = 147/1114*100
print('Percentage of the spam messages in test_set is {:.2f}%'.format(pct_spam_test_set))


Percentage of the non-spam messages in test_set is 86.80%
Percentage of the spam messages in test_set is 13.20%


I see that percentages of spam and non-spam in both the training and the test set are similar.

## Letter Case and Punctuation

To calculate all these probabilities, I'll first need to perform a bit of data cleaning to bring the data in a format that will allow me to extract easily all the information we need. Let's begin the data cleaning process by removing the punctuation and bringing all the words to lower case.

In [13]:
#Remove all the punctuation from the SMS column
training_set['SMS'] = training_set['SMS'].str.replace('\W', ' ')
training_set.head(10)

,Label,SMS
0,ham,Yep by the pretty sculpture
1,ham,Yes princess Are you going to make me moan
2,ham,Welp apparently he retired
3,ham,Havent
4,ham,I forgot 2 ask ü all smth There s a card on ...
5,ham,Ok i thk i got it Then u wan me 2 come now or...
6,ham,I want kfc its Tuesday Only buy 2 meals ONLY ...
7,ham,No dear i was sleeping P
8,ham,Ok pa Nothing problem
9,ham,Ill be there on lt gt ok


In [14]:
# transform every letter in every word to lower case
training_set['SMS'] = training_set['SMS'].str.lower()
training_set.head(10)

,Label,SMS
0,ham,yep by the pretty sculpture
1,ham,yes princess are you going to make me moan
2,ham,welp apparently he retired
3,ham,havent
4,ham,i forgot 2 ask ü all smth there s a card on ...
5,ham,ok i thk i got it then u wan me 2 come now or...
6,ham,i want kfc its tuesday only buy 2 meals only ...
7,ham,no dear i was sleeping p
8,ham,ok pa nothing problem
9,ham,ill be there on lt gt ok


## Creating the Vocabulary

The end goal with this data cleaning process is to bring our training set to this format:

![](https://dq-content.s3.amazonaws.com/433/cpgp_dataset_3.png)


First let's create a list with all of the unique words that occur in the messages of our training set

In [37]:
#create an empty list named 'vocabulary'

vocabulary = []

In [16]:
#transform each message from the SMS column into a list by
#splitting the string at the space character 

training_set['SMS'] = training_set['SMS'].str.split()

In [17]:
#using a nested loop, iterate each message in the SMS column  
#and append each string (word) to the vocabulary list

for row in training_set['SMS']:
    for word in row:
        vocabulary.append(word)

In [18]:
#transform the vocabulary list into a set using the set() function
#and than transform the vocabulary set back into a list using the list() function

vocabulary = list(set(vocabulary))

In [19]:
#total numbers of unique words in vocabulary
len(vocabulary)

7783

## The Final Training Set

I'm now going to use the vocabulary I just created to make the data transformation we want.

In [20]:
word_counts_per_sms = {unique_word: [0] * len(training_set['SMS']) for unique_word in vocabulary}

for index, sms in enumerate(training_set['SMS']):
    for word in sms:
        word_counts_per_sms[word][index] += 1
        

In [21]:
word_counts = pd.DataFrame(word_counts_per_sms)
word_counts.head()

,othrwise,yr,wellda,reasonable,hey,hop,08712405020,鈥,swhrt,school,...,plate,px3748,08719839835,2moro,spile,420,dismay,ne,tick,walkin
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [22]:
final_training_set = pd.concat([training_set, word_counts], axis=1)
final_training_set.head()

,Label,SMS,othrwise,yr,wellda,reasonable,hey,hop,08712405020,鈥,...,plate,px3748,08719839835,2moro,spile,420,dismay,ne,tick,walkin
0,ham,"[yep, by, the, pretty, sculpture]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,ham,"[yes, princess, are, you, going, to, make, me,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,ham,"[welp, apparently, he, retired]",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,ham,[havent],0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,ham,"[i, forgot, 2, ask, ü, all, smth, there, s, a,...",0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


## Calculating Constants First

Let's first calculate:

- P(Spam) and P(Ham)
- NSpam, NHam, NVocabulary


In [23]:
#recall the number of spam and ham messages in training_set:
#ham     3858
#spam     600
#total   4458

# P(Spam) = number of spam messages / total number of messages 

p_spam = 600/4458
p_spam

0.13458950201884254

In [24]:
# P(ham) = number of ham messages / total number of messages 

p_ham = 3858 / 4458
p_ham

0.8654104979811574

In [25]:
#NSpam is equal to the number of words in all the spam messages
spam_messages = final_training_set[final_training_set['Label'] == 'spam']

count_spam_messages = spam_messages.apply(len)
n_spam = count_spam_messages.sum()
n_spam

4671000

In [26]:
ham_messages = final_training_set[final_training_set['Label'] == 'ham']
ham_messages = final_training_set[final_training_set['Label'] == 'ham']
count_ham_messages = ham_messages.apply(len)
n_ham = count_ham_messages.sum()
n_ham

30034530

In [27]:
#N_Vocabulary
n_vocabulary = len(vocabulary)
print(n_vocabulary)


7783


In [28]:
#Laplace smoothing
alpha = 1

## Calculating Parameters

In more technical language, the probability values that P(wi|Spam) and P(wi|Ham) will take are called parameters.

Let's now calculate all the parameters using the equations below:



$$
P(w_i∣Spam) =  {N_{w_i}∣Spam + \alpha \above{2pt} N_{Spam} + \alpha * N_{Vocabulary}}
$$

$$
P(w_i∣Ham) =  {N_{w_i}∣Ham + \alpha \above{2pt} N_{Ham} + \alpha * N_{Vocabulary}}
$$


As P(wi|Spam) and P(wi|Ham) are key parts of the equations that we need to classify the new messages:

$$
P(Spam|w_1,w_2, \dots , w_n) ∝ P(Spam) ⋅ \displaystyle\prod_{i=1}^{n} P(w_i|Spam)
$$

$$
P(Ham|w_1,w_2, \dots , w_n) ∝ P(Ham) ⋅ \displaystyle\prod_{i=1}^{n} P(w_i|Ham)
$$







In [29]:
#Initialize two dictionaries, where each key-value pair is a unique word (from our vocabulary) represented as a string, and the value is 0
parameters_spam = {unique_word:0 for unique_word in vocabulary}
parameters_ham = {unique_word:0 for unique_word in vocabulary}


In [30]:
for word in vocabulary:
    n_word_given_spam = spam_messages[word].sum()
    p_word_given_spam = (n_word_given_spam + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_spam[word] = p_word_given_spam
    
    n_word_given_ham = ham_messages[word].sum()
    p_word_given_ham = (n_word_given_ham + alpha) / (n_ham + alpha*n_vocabulary)
    parameters_ham[word] = p_word_given_ham
    

## Classifying A New Message

The spam filter can be understood as a function that:

- Takes in as input a new message (w1, w2, ..., wn)
- Calculates P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn)
- Compares the values of P(Spam|w1, w2, ..., wn) and P(Ham|w1, w2, ..., wn), and:
    - If P(Ham|w1, w2, ..., wn) > P(Spam|w1, w2, ..., wn), then the message is classified as ham.
    - If P(Ham|w1, w2, ..., wn) < P(Spam|w1, w2, ..., wn), then the message is classified as spam.
    - If P(Ham|w1, w2, ..., wn) = P(Spam|w1, w2, ..., wn), then the algorithm may request human help.


In [31]:
import re

def classify(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham
    
    print('P(Spam|message):', p_spam_given_message)
    print('P(Ham|message):', p_ham_given_message)

    if p_ham_given_message > p_spam_given_message:
        print('Label: Ham')
    elif p_ham_given_message < p_spam_given_message:
        print('Label: Spam')
    else:
        print('Equal proabilities, have a human classify this!')

In [32]:
#classifying two first messages

classify('WINNER!! This is the secret code to unlock the money: C3421.')

P(Spam|message): 0.13458950201884254
P(Ham|message): 0.8654104979811574
Label: Ham


In [33]:
#classifying two second messages

classify("Sounds good, Tom, then see u there")

P(Spam|message): 0.13458950201884254
P(Ham|message): 0.8654104979811574
Label: Ham


## Measuring the Spam Filter's Accuracy

The algorithm will output a classification label for every message in our test set, which we'll be able to compare with the actual label (given by a human). Our algorithm didn't see these 1,114 messages, so every message in the test set is practically new from the perspective of the algorithm. 

Next we'll change the classify() function that we wrote previously to return the labels instead of printing them. 

In [34]:
def classify_test_set(message):

    message = re.sub('\W', ' ', message)
    message = message.lower()
    message = message.split()

    p_spam_given_message = p_spam
    p_ham_given_message = p_ham

    for word in message:
        if word in parameters_spam:
            p_spam_given_message *= parameters_spam[word]

        if word in parameters_ham:
            p_ham_given_message *= parameters_ham[word]

    if p_ham_given_message > p_spam_given_message:
        return 'ham'
    elif p_spam_given_message > p_ham_given_message:
        return 'spam'
    else:
        return 'needs human classification'

The returning labels we can use to create a new column in our test set


In [35]:
test_set['predicted'] = test_set['SMS'].apply(classify_test_set)
test_set.head()

,Label,SMS,predicted
0,ham,Later i guess. I needa do mcat study too.,ham
1,ham,But i haf enuff space got like 4 mb...,ham
2,spam,Had your mobile 10 mths? Update to latest Oran...,spam
3,ham,All sounds good. Fingers . Makes it difficult ...,ham
4,ham,"All done, all handed in. Don't know if mega sh...",ham


Now we can compare the predicted values with the actual values to measure how good our spam filter is with classifying new messages. To make the measurement, we'll use accuracy as a metric: 

$$
\text{Accuracy} =  {\text{number of correctly classified messages} \above{2pt} \text{total number of classified messages}}
$$



In [36]:
correct = 0
total = len(test_set)
    
for row in test_set.iterrows():
    row = row[1]
    if row['Label'] == row['predicted']:
        correct += 1
accuracy= round((correct/total)*100, 2)
print(accuracy)

94.08


In this project, we managed to build a spam filter for SMS messages using the multinomial Naive Bayes algorithm. The filter had an accuracy of 94.08% on the test set, which is an excellent result. We initially aimed for an accuracy of over 80%, but we managed to do way better than that.